In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
replacements = [",", ".", ":", '\'','\'', '"', "!", "?", "." ";", "@", "{", "}", "[", "]", "(", ")", "/", "#", "*", "_", "%", "-", "£", "$", "&"]
articles_prepositions = ['a', 'an', 'the', 'about', 'above', 'across', 'after', 'against', 'among', 'around',
'at', 'before', 'behind', 'below', 'beside', 'between', 'by', 'down', 'during', 'for', 'from', 'in', 'inside',
'into', 'near', 'of', 'off', 'on', 'out', 'over', 'through', 'to', 'toward', 'under', 'up', 'with']
articles = []
details = []
biases = []
for i in range(0,300):
        words = []
        counts = []
        isPunctuation = []
        articles.append('Article %d' % i)
        # Opening the article file, assigning text it's contents, and closing the file
        f = open("cleanedArticles/Article%d.txt" % i, 'r', encoding="utf-8")
        lines = f.readlines()
        f.close()

# reading the sanitized data from Article0 and turning it into a pandas dataframe
        for line in lines:
                data = line.split()
                newWord = data[0]
                newWord = newWord[0:(len(newWord)-1)]
                
                for exempt in articles_prepositions:
                        if(newWord==exempt):
                                continue

                if((len(newWord)==3) and (newWord[1:2] in letters)):
                        continue
                for r in range(0, len(replacements)):
                        if(len(newWord) > 3): 
                                newWord = newWord.replace(replacements[r], "")
                        
                for r in range(0, len(replacements)):
                        data[1] = data[1].replace(replacements[r], "")
                if(len(newWord)>2):
                        if(newWord[(len(newWord)-1):(len(newWord))] == 'i'):
                                newWord = newWord[0:(len(newWord)-1)]

                words.append(newWord)
                counts.append(data[1])
        g=0
        #implementing the isPunctuation functionality
        for word in words:
                isPunctuation.append('')

                if(len(word) == 3):
                        for letter in letters:
                                if(word.__contains__(letter)):
                                        isPunctuation[g] = ('False')
                        if(isPunctuation[g] != 'False'):
                                isPunctuation[g] = 'True'
                if(isPunctuation[g] == ''):
                        isPunctuation[g] = 'False'
                g=g+1
        newWords = []
        g=0
        for word in words:
                if(isPunctuation[g] == 'True'):
                        word = word[1:2]
                newWords.append(word)
                g+=1
        
        # storing every sentence in an article as a list
        sentences = []
        text = ""

        f = open("articles/article%d.txt" % i, 'r', encoding="utf-8")
        text = str(f.read()).lower()
        f.close()
        closingPunctuation = ['.', '?', '!']

        text = text.replace("\n", "")

        for r in range(0, len(closingPunctuation)):
            text = text.replace((closingPunctuation[r]), closingPunctuation[r] + "|")
            text = text.replace("| ", "|")

        sentences = text.split('|')

        mapping = []
        map = {}
        for word in newWords:
                sentenceIndex=0
                sentenceList = ""
                for sentence in sentences:
                        if(sentence.__contains__(word)):
                                sentenceList = sentenceList + str(sentenceIndex) + " "
                        sentenceIndex+=1
                sentenceList = sentenceList[0:(len(sentenceList)-1)]
                mapping.append(sentenceList)

        #finding words that are used in the same sentences
        sentences = []
        text = ""
        f = open("articles/article0.txt", 'r', encoding="utf-8")
        text = str(f.read()).lower()
        f.close()
        closingPunctuation = ['.', '?', '!']

        text = text.replace("\n", "")

        for r in range(0, len(closingPunctuation)):
                text = text.replace((closingPunctuation[r]), closingPunctuation[r] + "|")
                text = text.replace("| ", "|")

        sentences = text.split('|')


        # it works!

        deets = []
        terms = []
        sentenceGroups = []
        sentenceInfo =[]
        terms = newWords
        #logic here is a little sloppy, could be condensed
        for term in terms:
                sentenceGroups=[]
                sentenceID = 0
                for sentence in sentences:
                        wordsInSentence= []
                        wordsTogether = []
                        if(sentence.__contains__(term)):
                                wordsInSentence = sentence.split()
                                for potentialWord in wordsInSentence:
                                        if(potentialWord!=term):
                                                for replacee in replacements:
                                                        potentialWord = potentialWord.replace(replacee, '')
                                                wordsTogether.append(potentialWord)
                        else:
                                continue
                        sentenceGroups.append(wordsTogether)
                sentenceInfo.append(sentenceGroups)
        g = open("articleBiases/article%dBias.txt" % i, 'r', encoding="utf-8")
        bias = g.read()
        g.close()
        if(bias.__contains__('-1')):
                biases.append('-1')
        else:
                biases.append('1')
        
        details.append({'Words': newWords, 'Counts':counts, 'isPunctuation': isPunctuation, 'sentenceMap': mapping, 'Associations': sentenceInfo})
        
dfnew = pd.DataFrame(details)
#dfnew['Article'] = articles
dfnew['Bias'] = biases

dfnew

Moving onto the tensorflow classification

In [2]:
import pandas as pd
replacements = [",", ".", ":", "'",'\'', '—', '"', '\'' "!", "?", "." ";", "@", "{", "}", "[", "]", "(", ")", "/", "#", "*", "_", "%", "-", "£", "$", "&"]
articles_prepositions = ['a', 'an', 'the', 'about', 'above', 'across', 'after', 'against', 'among', 'around',
'at', 'before', 'behind', 'below', 'beside', 'between', 'by', 'down', 'during', 'for', 'from', 'in', 'inside',
'into', 'near', 'of', 'off', 'on', 'out', 'over', 'through', 'to', 'toward', 'under', 'up', 'with']
texts = []
biases = []
data = {}
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
for i in range(0,300):
    f = open("articles/article%d.txt" % i, 'r', encoding="utf-8")
    contents = f.read().lower()
    f.close()

    contents = contents.replace("CNN", '')
    contents = contents.replace("cnn", '')
    contents = contents.replace("Advertisement", '')
    contents = contents.replace("fox", '')
    contents = contents.replace("FOX", '')
    contents = contents.replace("huffpost", '')
    contents = contents.replace("washingtonpost", '')
    contents = contents.replace("Washington Post", '')
    contents = contents.replace("The Federalist", '')
    contents = contents.replace("\n", '')
    for r in range(0, len(replacements)):
        contents = contents.replace(replacements[r], '')

    for ap in range(0, len(articles_prepositions)):
        contents = contents.replace((' ' +articles_prepositions[ap] + ' '), ' ')
    
    stuff = contents.split()
    filtered_stuff = [word for word in stuff if word not in stopwords.words('english')]
    contents = ''
    for word in filtered_stuff:
        word = ps.stem(word)
        contents = contents + word + ' '

    texts.append(contents)
    
    g = open("articleBiases/article%dBias.txt" % i, 'r', encoding='utf-8')
    bias = g.read()
    g.close()
    biases.append(bias)

data = {'Article': texts, 'Bias': biases}

df = pd.DataFrame(data)

df = df.dropna()
df.isnull().sum()

df



In [1]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
#opt = SGD(lr=0.01)
#tokenizer = Tokenizer(oov_token="<OOV>")


#splitting data into a training and test dataset
split = round(len(df)*0.8)
train_reviews = df['Article'][:split]
train_label = df['Bias'][:split]
test_reviews = df['Article'][split:]
test_label = df['Bias'][split:]

#sanitizing all data as a string
import numpy as np
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_reviews:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append(int(row))
for row in test_reviews:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append(int(row))

# variable constants
vocab_size = 40000
embedding_dim = 16
max_length =  1000
trunc_type = 'post'
#oov_tok = '<OOV>'
padding_type = 'post'

# tokenizing 
    # words
#tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

    # sentences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,       input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    #tf.keras.layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)),
    #tf.keras.layers.LSTM(15,dropout=0.5),
    #tf.keras.layers.MaxPooling1D(5),
    #tf.keras.layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)),
    #tf.keras.layers.GlobalMaxPooling1D(),
    #tf.keras.layers.Dense(3, input_dim=1, activation='relu'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

#training 

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)

num_epochs = 50
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final), callbacks=[checkpoint1])

In [ ]:
training_sentences